In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars.packages":"org.apache.iceberg:iceberg-spark3-runtime:0.11.0",
             "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
             "spark.sql.catalog.spark_catalog":"org.apache.iceberg.spark.SparkSessionCatalog",
             "spark.sql.catalog.spark_catalog.type":"hive",
             "spark.sql.catalog.local":"org.apache.iceberg.spark.SparkCatalog",
             "spark.sql.catalog.local.type":"hadoop",
             "spark.sql.catalog.local.warehouse":"s3://vasveena-test-demo/iceberg/catalog/tables/"
           } 
}

In [2]:
spark.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1635758941805_0019,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: String = 3.0.1-amzn-0


In [3]:
val input_df = spark.read.parquet("s3://vasveena-test-demo/tmp/hudi-perf/input/")
input_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 5 more fields]
res2: Long = 2700000000


In [4]:
input_df.createOrReplaceTempView("inputdf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import org.apache.spark.sql.functions._

val input_df2=(input_df.withColumn("z", substring(md5(concat($"id")),1,1))
                       .withColumn("schema-v", lit("v1")).withColumn("data-v", lit("v2"))
                       .withColumn("trade_dt", substring($"modified_timestamp",1,10)))
input_df2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
input_df2: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema-v|data-v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000000|    3|4000000|[E]|6e505939-f5fd-4ab...|2019|2021-04-02 00:05:02|  9|      v1|    v2|2021-04-02|
|4000001|    9|4000001|[F]|20486aca-2759-43f...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000002|   11|4000002|[G]|42962a21-a2dc-40d...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000003|    9|4000003|[H]|9841ad6d-1532-496...|2019|2021-04-02 00:05:02|  c|      v1|    v2|2021-04-02|
|4000004|    4|4000004|[I]|ff1a855a-cced-495...|2019|2021-04-02 00:05:02|  4|      v1|    v2|2021-04-02|
+-------+-----+-------+---+---

In [6]:
val input_df4 = input_df2.withColumnRenamed("schema-v", "schema_v").withColumnRenamed("data-v", "data_v")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df4: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]


In [7]:
input_df4.registerTempTable("inputdf4")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
spark.sql("show tables in local.db").show(false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------------+
|namespace|tableName                 |
+---------+--------------------------+
|db       |iceberg_table_sparksqldf28|
|db       |iceberg_table_sparksqldf26|
|db       |iceberg_table_sparksqldf24|
|db       |iceberg_table_sparksqldf22|
+---------+--------------------------+



In [9]:
input_df4.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- month: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: struct (nullable = true)
 |    |-- key1: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- year: string (nullable = true)
 |-- modified_timestamp: timestamp (nullable = true)
 |-- z: string (nullable = true)
 |-- schema_v: string (nullable = false)
 |-- data_v: string (nullable = false)
 |-- trade_dt: string (nullable = true)



In [18]:
spark.sql("drop table local.db.iceberg_table_sparksqldf26")
spark.sql("drop table local.db.iceberg_table_sparksqldf28")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res17: org.apache.spark.sql.DataFrame = []
res18: org.apache.spark.sql.DataFrame = []


In [19]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf26 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.object-storage.path'='s3://vasveena-test-hmswh')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf26'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res19: org.apache.spark.sql.DataFrame = []


In [8]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf28 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.object-storage.path'='s3://vasveena-test-hmswh')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf28'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res7: org.apache.spark.sql.DataFrame = []


In [21]:
val t1 = System.nanoTime
input_df4.drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").writeTo("local.db.iceberg_table_sparksqldf26").overwritePartitions()
input_df4.printSchema()
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 1640578607171
root
 |-- id: long (nullable = true)
 |-- month: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: struct (nullable = true)
 |    |-- key1: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- year: string (nullable = true)
 |-- modified_timestamp: timestamp (nullable = true)
 |-- z: string (nullable = true)
 |-- schema_v: string (nullable = false)
 |-- data_v: string (nullable = false)
 |-- trade_dt: string (nullable = true)

duration: String = 448.554604046seconds


In [23]:
spark.sql("select count(*) from local.db.iceberg_table_sparksqldf26").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|  count(1)|
+----------+
|2700000000|
+----------+



In [10]:
val t1 = System.nanoTime
input_df4.repartition(col("modified_timestamp"),col("month")).drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").writeTo("local.db.iceberg_table_sparksqldf28").overwritePartitions()
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 689674372673
duration: String = 967.207176442seconds


In [12]:
spark.sql("select count(*) from local.db.iceberg_table_sparksqldf28").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|  count(1)|
+----------+
|2700000000|
+----------+



In [9]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf32 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.object-storage.path'='s3://vasveena-test-hmswh')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf32'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res8: org.apache.spark.sql.DataFrame = []


In [14]:
val t1 = System.nanoTime
input_df4.drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").writeTo("local.db.iceberg_table_sparksqldf32").overwritePartitions()
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 1940980666449
duration: String = 499.766054749seconds


In [10]:
val t1 = System.nanoTime
(input_df4
 //.repartition(col("modified_timestamp"),col("month"))
 .repartition(col("z"),col("schema_v"),col("data_v"),col("trade_dt")).drop(col("month"))
 .drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt")
 .writeTo("local.db.iceberg_table_sparksqldf28").overwritePartitions())
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 300877900568288
duration: String = 1477.762195897seconds


In [11]:
val t1 = System.nanoTime
(input_df4
 .drop(col("month"))
 .sortWithinPartitions("z","schema_v","data_v","trade_dt")
 .writeTo("local.db.iceberg_table_sparksqldf32").overwritePartitions())
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 302357177807241
duration: String = 500.021154291seconds


In [12]:
spark.sql("""select sum(file_size_in_bytes)/1024/1024/1024 as SizeGB from local.db.iceberg_table_sparksqldf32.files""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|           SizeGB|
+-----------------+
|64.78852318879217|
+-----------------+



In [13]:
spark.sql("""select sum(file_size_in_bytes)/1024/1024/1024 as SizeGB from local.db.iceberg_table_sparksqldf28.files""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|           SizeGB|
+-----------------+
|65.38351830560714|
+-----------------+

